check performance of "batch solve" for parallel solutions

issue was Node.AddAsDependency not adding parents

lineage constraint used to guide the dfs

In [9]:
from pathlib import Path
from dataclasses import dataclass
from metasmith import DataInstanceLibrary, Endpoint, DataTypeLibrary, TransformInstanceLibrary
from metasmith.hashing import KeyGenerator

from local.constants import WORKSPACE_ROOT
from local.utils import LinkifyPath
CACHE = WORKSPACE_ROOT/"main/local_mock/cache"

In [30]:
TEST_DIR = CACHE/"test_1k_parallel"
TEST_DIR.mkdir(parents=True, exist_ok=True)

types = DataTypeLibrary.Load(WORKSPACE_ROOT/"main/local_mock/prototypes/metagenomics.dev3.yml")
refdb = DataInstanceLibrary.Load(CACHE/"ref.xgdb")

xgdbs: list[DataInstanceLibrary] = []
for i in range(1000):
    xgdb = DataInstanceLibrary(TEST_DIR/f"{i:03}.xgdb")
    _type = types["contigs"].Clone(properties_only=True)
    _type.AddProperty(key="id", value=f"{i}")
    xgdb.ImportDataInstance(WORKSPACE_ROOT/"scratch/test_ws/data/local/example.fna", _type, overwrite=True)
    xgdbs.append(xgdb)

In [31]:
trlib = TransformInstanceLibrary.Load([
    Path("./transforms/simple_1"),
    # Path("./transforms/dupe_test"),
])
for k, v in trlib:
    print(k.name, type(v))

diamond.py <class 'metasmith.models.libraries.TransformInstance'>
pprodigal.py <class 'metasmith.models.libraries.TransformInstance'>


In [32]:
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable

from metasmith.models.workflow import WorkflowStep
from metasmith.models.libraries import DataTypeLibrary
from metasmith.models.libraries import DataInstance, DataInstanceLibrary
from metasmith.models.libraries import TransformInstance, TransformInstanceLibrary
from metasmith.models.solver import Endpoint, Dependency, Transform, _solve_by_bounded_dfs

# plan = WorkflowPlan.Generate(
#     given=xgdbs+[refdb],
#     transforms=trlib,
#     # targets=[
#     #     types["orf_annotations"] for i, xgdb in enumerate(xgdbs)
#     # ],
#     targets=[
#         types["orf_annotations"].WithLineage([
#             # xgdb["example.fna"].type,
#         ]) for i, xgdb in enumerate(xgdbs)
#     ],
# )
    
given: Iterable[DataInstanceLibrary]=xgdbs+[refdb]
transforms: TransformInstanceLibrary=trlib
targets: list[Endpoint]=[
    types["orf_annotations"].WithLineage([
        xgdb["example.fna"].type,
    ]) for i, xgdb in enumerate(xgdbs)
]

_given_map: dict[Endpoint, DataInstance] = {}
for lib in given:
    _map = {e.type:e for e in lib.manifest.values()}
    for inst in _map.values():
        inst.source = lib.source/inst.source
    _given_map.update(_map)

_transform_map = {t.model:t for _, t in transforms}

_target_e2d: dict[Endpoint, Dependency] = {}
_target_model = Transform()
for t in targets:
    t.AddAsDependency(_target_model, _target_e2d)

def do():
    return _solve_by_bounded_dfs(
        given=_given_map.keys(),
        target=_target_model,
        transforms=_transform_map.keys(),
        # _debug=True,
    )
%prun solutions = do()
# solutions = do()

assert len(solutions) > 0, "failed to make plan!"
solution = solutions[0]

_instance_map = _given_map.copy()
steps: list[WorkflowStep] = []
for i, appl in enumerate(solution.dependency_plan):
    tr = _transform_map[appl.transform]
    for e, d in appl.produced.items():
        p = tr.output_signature[d]
        _instance = DataInstance(p, e)
        _instance_map[e] = _instance
    
    step = WorkflowStep(
        order=i+1,
        transform_key=tr.model.key,
        uses=[_instance_map[e] for e in appl.used],
        produces=[_instance_map[e] for e in appl.produced],
        transform=tr,
    )
    steps.append(step)

_sol_produces_d2e = {d:e for e, d in solution.application.used.items()}
_sol_target_instances: list[DataInstance] = []
for e in targets:
    d = _target_e2d[e]
    _appl_e = _sol_produces_d2e[d]
    _inst = _instance_map[_appl_e]
    _sol_target_instances.append(_inst)
    
for step in steps:
    if step.transform._source.name != "diamond.py": continue
    # print([f"{x.type}" for x in step.uses], [f"{x.type}" for x in step.produces], sep="->")
    # print([f"{x.source.name}:{x.type.key}" for x in step.uses], [f"{x.source.name}" for x in step.produces], sep="->")
    # LinkifyPath(step.transform._source)

         34326343 function calls (34308322 primitive calls) in 8.421 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  6000000    1.753    0.000    4.096    0.000 solver.py:384(<genexpr>)
  8019025    1.372    0.000    2.207    0.000 solver.py:75(IsA)
  8019025    0.835    0.000    0.835    0.000 {method 'issubset' of 'set' objects}
  6020000    0.823    0.000    0.940    0.000 solver.py:192(Iterparents)
     1003    0.744    0.001    0.756    0.001 solver.py:216(AddProduct)
  1001000    0.708    0.000    6.195    0.000 solver.py:423(_add_result)
  1004000    0.630    0.000    4.726    0.000 {built-in method builtins.all}
2005/2000    0.491    0.000    7.392    0.004 solver.py:388(_solve_dep)
  1006002    0.428    0.000    5.154    0.000 solver.py:382(_satisfies_lineage)
   5002/3    0.152    0.000    0.227    0.076 solver.py:482(_gather)
  1022023    0.118    0.000    0.118    0.000 {method 'items' of 'dict' objects}
    1